Importamos la librerias a utilizar

El objetivo final al que hemos llegado es desarrollar un modelo de Machine Learning que pueda predecir la demanda de taxis en la ciudad de Nueva York. Hemos pasado por todas las etapas del proceso de desarrollo de un modelo de principio a fin, desde la preparación de los datos hasta el despliegue del modelo en un entorno de producción. Ahora, tenemos la capacidad de hacer predicciones en tiempo real sobre la demanda de taxis, lo que puede ser extremadamente útil para los conductores de taxis y las empresas de transporte.

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Cargamos el archivo y visualizamos las primeras filas

In [61]:
taxis = pd.read_parquet('D:/escritorio/NYC-TAXIS/NYC_TAXIS-CO2/Data/Normalized/taxis_2023.parquet')
taxis.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough
0,2023-01-01 00:26:10,2023-01-01 00:37:11,1.0,2.58,24.18,Green,Manhattan
1,2023-01-01 00:51:03,2023-01-01 00:57:49,1.0,1.81,15.84,Green,Manhattan
2,2023-01-01 00:35:12,2023-01-01 00:41:32,1.0,0.00,11.64,Green,Queens
3,2023-01-01 00:13:14,2023-01-01 00:19:03,1.0,1.30,10.20,Green,Manhattan
4,2023-01-01 00:33:04,2023-01-01 00:39:02,1.0,1.10,8.00,Green,Manhattan


In [62]:
# Modificamos el DataFrame original para asignar valores numéricos a los tipos de servicio
taxis['Servicio'] = taxis['Servicio'].map({'Yellow': 1, 'Green': 2})

Conversión de Fechas

In [63]:
taxis['tpep_pickup_datetime'] = pd.to_datetime(taxis['tpep_pickup_datetime'])
taxis['tpep_dropoff_datetime'] = pd.to_datetime(taxis['tpep_dropoff_datetime'])

Ingeniería de Características

In [64]:
taxis['pickup_hour'] = taxis['tpep_pickup_datetime'].dt.hour
taxis['pickup_day'] = taxis['tpep_pickup_datetime'].dt.dayofweek

taxisnyc = taxis[['pickup_hour', 'pickup_day', 'Servicio', 'Pickup_borough']]

Visualización de Datos

In [65]:
taxisnyc.head(2)

,pickup_hour,pickup_day,Servicio,Pickup_borough
0,0,6,2,Manhattan
1,0,6,2,Manhattan


Agrupación de Datos

In [66]:
demand_data = taxisnyc.groupby(['Pickup_borough', 'Servicio', 'pickup_day', 'pickup_hour']).size().reset_index(name='demand')
demand_data.head(3)

,Pickup_borough,Servicio,pickup_day,pickup_hour,demand
0,Brooklyn,1,0,0,17
1,Brooklyn,1,0,1,16
2,Brooklyn,1,0,2,11


Codificación de Variables Categóricas

In [67]:
from sklearn.preprocessing import LabelEncoder

le_borough = LabelEncoder()
le_service = LabelEncoder()

demand_data['Pickup_borough'] = le_borough.fit_transform(demand_data['Pickup_borough'])
demand_data['Servicio'] = le_service.fit_transform(demand_data['Servicio'])

Dividimos Datos de Entrenamiento y de Pruebas

In [68]:
from sklearn.model_selection import train_test_split

X = demand_data[['Pickup_borough', 'Servicio', 'pickup_day', 'pickup_hour']]
y = demand_data['demand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Entrenemiento del Modelo

In [69]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Evaluación del Modelo Original

In [70]:
from sklearn.metrics import mean_squared_error

rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest MSE: {rf_mse}')


Random Forest MSE: 81264.58294319999


 Optimización del Modelo

In [71]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

rf_model = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)


rf_model_best = RandomForestRegressor(**best_params, random_state=42)
rf_model_best.fit(X_train, y_train)

rf_pred_best = rf_model_best.predict(X_test)

rf_mse_best = mean_squared_error(y_test, rf_pred_best)
print(f'Random Forest MSE (mejorado): {rf_mse_best}')

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=150; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=150; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=150; total time=   0.4s
[CV] END max_depth=None, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_sa

Guardar el Modelo Optimizado

In [72]:
import joblib
joblib.dump(rf_model_best, 'random_forest_model.pkl')

['random_forest_model.pkl']

Cargar el Modelo en Producción

In [73]:
loaded_model = joblib.load('random_forest_model.pkl')

Hacer Predicciones en Tiempo Real (Ejemplo 1)

In [74]:
new_data = [[1, 8, 16, 1]]  
prediction = loaded_model.predict(new_data)
print(f'Predicción de demanda: {prediction}')

Predicción de demanda: [45.26]


C:\Users\lautaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Hacer Predicciones en Tiempo Real (Ejemplo 2)

In [75]:
new_data = [[0, 6, 10, 2]] 
prediction = loaded_model.predict(new_data)
print(f'Predicción de demanda: {prediction}')

Predicción de demanda: [839.]


C:\Users\lautaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


En el ejemplo que proporcione, los números representan las características que se utilizarán como entrada para hacer la predicción de la demanda de taxis.

El primer valor, 0, representa la hora del día. En este caso, 0 podría representar la medianoche o las 12:00 a. m.

El segundo valor, 6, representa el día de la semana. Los días de la semana se numeran del 0 al 6, donde 0 es el lunes y 6 es el domingo.

El tercer valor, 10, parece ser el distrito de recogida, esta sea una codificación numérica para un distrito específico de la ciudad de Nueva York.

El cuarto valor, 1, podría ser otra característica relevante, relacionada con el tipo de servicio de taxi

Por lo tanto, estos números representan las características que el modelo utilizará para hacer la predicción de la demanda de taxis en la ciudad de Nueva York en un momento específico. El valor devuelto, aproximadamente 839. sería la predicción de la cantidad de taxis necesarios en esas condiciones.